In [ ]:
# 토큰 정보로드를 위한 라이브러리
# 설치: pip install python-dotenv
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

## 1. 오디오파일에 자막생성
- response_format="srt"


In [ ]:
from openai import OpenAI

client = OpenAI()

In [ ]:
audio_file = open("data/채용면접_샘플_01.wav", "rb")
transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    language="ko",
    response_format="srt",  # 자막 포맷
    temperature=0.0,
)

In [ ]:
print(transcript)

## 2. YouTube url에서 음성파일을 만들기


In [ ]:
from pytube import YouTube
import os

link = "https://www.youtube.com/watch?v=ZsQX7x_KWjo&t=2851s"

yt = YouTube(link)
filename = yt.streams.filter(only_audio=True).first().download()
renamed_file = filename.replace(".mp4", ".mp3")
os.rename(filename, renamed_file)

In [ ]:
print(renamed_file)

## 3 MP3를  WAV 변환


In [ ]:
from moviepy.editor import AudioFileClip


def convert_mp3_to_wav(filepath):
    # WAV 파일 경로
    wav_file_path = filepath.replace(".mp3", ".wav")

    # MP4 파일 로드
    audio_clip = AudioFileClip(filepath)

    # WAV 형식으로 오디오 추출 및 저장
    audio_clip.write_audiofile(wav_file_path, fps=44100, nbytes=2, codec="pcm_s16le")
    return wav_file_path

In [ ]:
wav_file = convert_mp3_to_wav(renamed_file)

In [ ]:
from pydub import AudioSegment

# 오디오 파일 불러오기
audio = AudioSegment.from_file(wav_file, format="wav")

total_length = len(audio)
length_per_chunk = 60 * 1000  # 60초

if not os.path.exists(".tmp"):
    os.mkdir(".tmp")

folder_path = os.path.join(".tmp", wav_file[:-4])

if not os.path.exists(folder_path):
    os.mkdir(folder_path)

chunks = []
for i in range(0, total_length, length_per_chunk):
    chunk_file_path = os.path.join(folder_path, f"{i}.wav")
    audio[i : i + length_per_chunk].export(chunk_file_path, format="wav")
    chunks.append(chunk_file_path)

In [ ]:
chunks

In [ ]:
from datetime import datetime, timedelta
import re


def adjust_timestamps(transcript, minutes=1):
    # Define the regular expression pattern for timestamps
    timestamp_pattern = re.compile(r"(\d{2}:\d{2}:\d{2},\d{3})")

    # Function to add minutes to a timestamp
    def add_minutes(timestamp_str, minutes):
        timestamp = datetime.strptime(timestamp_str, "%H:%M:%S,%f")
        adjusted_timestamp = timestamp + timedelta(minutes=minutes)
        return adjusted_timestamp.strftime("%H:%M:%S,%f")[:-3]

    # Replace timestamps in the transcript
    adjusted_transcript = timestamp_pattern.sub(
        lambda match: add_minutes(match.group(1), minutes), transcript
    )
    return adjusted_transcript

In [ ]:
transcripts = []
for i, chunk in enumerate(chunks):
    print(chunk)
    audio_file = open(chunk, "rb")
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        language="ko",
        response_format="srt",
        temperature=0.01,
    )
    transcripts.append(adjust_timestamps(transcript, minutes=i))

In [ ]:
def merge_transcripts(*transcripts):
    merged_transcript = ""
    current_number = 1

    for transcript in transcripts:
        # Split the transcript into segments
        segments = transcript.strip().split("\n\n")
        for segment in segments:
            # Split each segment into lines
            lines = segment.split("\n")
            # Replace the number at the beginning of each segment with the correct sequence number
            lines[0] = str(current_number)
            # Increment the sequence number
            current_number += 1
            # Reassemble the segment
            merged_transcript += "\n".join(lines) + "\n\n"

    return merged_transcript.strip()

In [ ]:
print(merge_transcripts(*transcripts))

In [ ]:
merged_transcript = merge_transcripts(*transcripts)
print(merged_transcript)

In [ ]:
with open("sample.srt", "w") as f:
    f.write(merged_transcript)

In [ ]:
import tiktoken
from openai import OpenAI
import re

client = OpenAI()

MAX_COMPLETION_TOKENS = 4096
MAX_CONTEXT_LENGTH = 8192  # GPT-4의 일반적인 컨텍스트 길이

def count_tokens(text: str) -> int:
    encoding = tiktoken.encoding_for_model("gpt-4o")
    return len(encoding.encode(text))

def chunk_text(text: str, max_chunk_tokens: int = 3000) -> list[str]:
    chunks = []
    current_chunk = ""
    current_tokens = 0
    
    for sentence in text.split(". "):
        sentence_tokens = count_tokens(sentence)
        if current_tokens + sentence_tokens > max_chunk_tokens:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
            current_tokens = sentence_tokens
        else:
            current_chunk += sentence + ". "
            current_tokens += sentence_tokens
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

def process_chunk(chunk: str, system_prompt: str) -> str:
    prompt_tokens = count_tokens(system_prompt) + count_tokens(chunk)
    max_response_tokens = min(MAX_COMPLETION_TOKENS, MAX_CONTEXT_LENGTH - prompt_tokens - 100)

    if max_response_tokens <= 0:
        return "Error: 입력 텍스트가 너무 깁니다."
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            temperature=0.1,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": chunk},
            ],
            max_tokens=max_response_tokens,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: API 호출 중 오류 발생 - {str(e)}"

def format_summary(summary: str) -> str:
    lines = summary.split('\n')
    formatted_lines = []
    for line in lines:
        if line.strip():
            match = re.match(r'^(\d{2}:\d{2})\s+(.+)$', line)
            if match:
                time, content = match.groups()
                if not any(emoji in content for emoji in ['🔶', '👋', '👀', '💡', '🎯', '📊', '🔑', '💻', '🤔', '📝']):
                    content = '🔶 ' + content
                formatted_lines.append(f"{time} {content}")
            else:
                formatted_lines.append(line)
    return "\n".join(formatted_lines)

def post_processing(instruction: str) -> str:
    system_prompt = """
    당신은 비디오 자막을 분석하고 요약하는 AI 어시스턴트입니다. 주어진 자막 정보를 바탕으로 다음 작업을 수행해야 합니다:
    1. 중요한 주제를 시간 순서에 맞게 선정하세요.
    2. 각 주제에 대해 한 줄 요약을 작성하세요.
    3. 각 주제가 시작되는 시간을 mm:ss(분:초) 형식으로 기록하세요.
    4. 각 요약 문장에 적합한 이모지를 추가하세요.
    5. 요약은 한글로 작성하세요.
    6. 주제는 최대한 많이 추출해 주세요.
    7. 시간 표기에 오류가 없도록 주의하세요.

    출력 형식:
    mm:ss 🔶 주제에 대한 한 줄 요약
    """

    chunks = chunk_text(instruction)
    summaries = []

    print(f"총 {len(chunks)}개의 청크로 나누어 처리합니다.\n")

    for i, chunk in enumerate(chunks):
        print(f"\n청크 {i+1}/{len(chunks)} 처리 중...\n")
        summary = process_chunk(chunk, system_prompt)
        summaries.append(summary)
        print(summary)
        print("\n")

    final_summary = "\n".join(summaries)
    formatted_summary = format_summary(final_summary)

    return formatted_summary



In [ ]:
instruction = f"""주어진 비디오 "자막정보" 를 바탕으로 [요청사항]을 차례대로 수행해주세요.
[자막정보]
{merged_transcript}

[요청사항]
1. 주어진 [자막정보]에서 중요한 주제를 시간 순서에 맞게 선정하고, 주제가 시작되는 시간을 기록해주세요.
2. 주제는 한 줄 요약을 작성하고, 자막에서 주제가 시작되는 시간을 mm:ss(분:초) 형식으로 작성하세요. (예: 00:05 👋 자막 생성 기능에 대한 소개)
3. 요약은 한글로 작성해주세요.
4. 각 문장에 적합한 emoji를 최대한 활용해 주세요

[출력예시]
00:12 👋 GPTs 의 주요 개념에 대하여 소개해요
03:13 👀 GPTs 의 장단점과 활용 사례에 대하여 자세히 알아봐요

[주의사항]
- 주제는 최대한 많이 추출해 주세요.
- 시간표기에 오류가 없도록 주의해 주세요. 분:초 형식으로 작성합니다. (예: 00:12)

run step-by-step. Take a deep breath. You can do it!
"""
summary_output = post_processing(instruction)

In [ ]:
print(summary_output)